# NB GLM test

In [22]:
import multiprocessing
import warnings
from math import floor
from pathlib import Path
from typing import List
from typing import Literal
from typing import Optional
from typing import Tuple
from typing import Union
from typing import cast

In [23]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.linalg import solve  # type: ignore
from scipy.optimize import minimize  # type: ignore
from scipy.special import gammaln  # type: ignore
from scipy.special import polygamma  # type: ignore
from scipy.stats import norm  # type: ignore
from sklearn.linear_model import LinearRegression  # type: ignore

In [24]:
#generate count data for a single gene (vector), sample 
counts = [511, 1783, 241, 1129, 1302, 2204, 3888, 5035, 236, 468, 1424, 482, 842, 1145, 1261, 1661, 2712, 1707, 1125, 3832]
counts = np.array(counts)
counts 

array([ 511, 1783,  241, 1129, 1302, 2204, 3888, 5035,  236,  468, 1424,
        482,  842, 1145, 1261, 1661, 2712, 1707, 1125, 3832])

In [25]:
# simulate CN-induced differential gene expression
cnv = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 5, 3, 7, 4, 8, 6, 4, 6, 5]
cnv = np.array(cnv)
cnv = cnv/2
counts = counts*cnv
counts

array([ 511. , 1783. ,  241. , 1129. , 1302. , 2204. , 3888. , 5035. ,
        236. ,  468. , 2848. , 1205. , 1263. , 4007.5, 2522. , 6644. ,
       8136. , 3414. , 3375. , 9580. ])

In [26]:
#generate CNV data
cnv = cnv + 10e-6
cnv

array([1.00001, 1.00001, 1.00001, 1.00001, 1.00001, 1.00001, 1.00001,
       1.00001, 1.00001, 1.00001, 2.00001, 2.50001, 1.50001, 3.50001,
       2.00001, 4.00001, 3.00001, 2.00001, 3.00001, 2.50001])

In [27]:
#Generate dispersion data
disp = 0.6037885 
alpha = disp
alpha

0.6037885

In [28]:
#Generate real calculated sf data
size_factors = np.array([1.16950695, 1.08637265, 1.15012341, 1.19435402, 1.08791923, 1.17677969, 1.16581881,
       0.77746277, 1.17488973, 0.91116436, 0.8980075 , 1.06022032, 0.90552191, 0.9371684 , 0.78716591, 0.99839825, 1.07677969,
                        0.9980075, 0.7980075, 1.05791923])
size_factors

array([1.16950695, 1.08637265, 1.15012341, 1.19435402, 1.08791923,
       1.17677969, 1.16581881, 0.77746277, 1.17488973, 0.91116436,
       0.8980075 , 1.06022032, 0.90552191, 0.9371684 , 0.78716591,
       0.99839825, 1.07677969, 0.9980075 , 0.7980075 , 1.05791923])

In [56]:
#Generate design matrix
#X = np.array([1, 0])
#X = np.repeat(X, [8, 8], axis=0)
#X.T

In [29]:
X = {'condition': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
X = pd.DataFrame(X, index = ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8', 'sample9', 'sample10',
                            'sample11', 'sample12', 'sample13', 'sample14', 'sample15', 'sample16', 'sample17', 'sample18', 'sample19', 'sample20'])
X.insert(0, "intercept", 1)

In [30]:
X = np.array(X)
X

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [31]:
design_matrix = X
num_vars = design_matrix.shape[1]
num_vars

2

In [11]:
Q, R = np.linalg.qr(X)
Q, R

(array([[-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068, -0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068],
        [-0.2236068,  0.2236068]]),
 array([[-4.47213595, -2.23606798],
        [ 0.        ,  2.23606798]]))

In [54]:
# Classical GLM
y = np.log(counts / size_factors + 0.1)
y

array([ 6.08001618,  7.40326924,  5.3454048 ,  6.85158788,  7.08747347,
        7.53530093,  8.11225647,  8.77590377,  5.30315522,  6.24169495,
        8.75511208,  7.95208404,  7.64600183,  9.61358482,  8.76528654,
        9.20854763, 10.02869582,  8.83079619,  9.44840806, 10.02742403])

In [14]:
beta_init = solve(R, Q.T @ y)
beta = beta_init
beta

array([6.87565277, 1.20850591])

In [18]:
ridge_factor = np.diag(np.repeat(1e-6, num_vars))
ridge_factor

array([[1.e-06, 0.e+00],
       [0.e+00, 1.e-06]])

In [19]:
# Classical model
min_mu = 0.5
mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
mu

array([1132.55907365, 1052.05121024, 1113.78791192, 1156.62115775,
       1053.54892961, 1139.6020482 , 1128.98745194,  752.90062585,
       1137.77179713,  882.3782223 , 2911.96042483, 3437.96640166,
       2936.32733105, 3038.94710479, 2552.53544953, 3237.49656013,
       3491.6633141 , 3236.2294788 , 2587.69137086, 3430.50467889])

In [20]:
# Analitical IRLS algorithm calculations check
W = mu / (1.0 + mu * disp)
W

array([0.74287877, 0.74284148, 0.74287056, 0.74288891, 0.74284223,
       0.74288178, 0.74287723, 0.74263314, 0.742881  , 0.74274064,
       0.74317665, 0.74320567, 0.74317822, 0.74318457, 0.74314994,
       0.74319572, 0.74320814, 0.74319565, 0.74315288, 0.74320532])

In [21]:
z = np.log(mu / size_factors) + (counts - mu) / mu
z

array([7.9108667 , 9.01428325, 8.94625532, 7.09904384, 6.45274984,
       6.04413263, 6.08646113, 9.02480675, 7.14216333, 6.38450486,
       9.22292401, 9.65835486, 7.53574321, 8.13978745, 7.87161101,
       8.74423766, 8.25666577, 7.6972178 , 7.42268437, 8.28150437])

In [22]:
H = (X.T * W) @ X + ridge_factor
H

array([[14.86018949,  7.43185276],
       [ 7.43185276,  7.43185376]])

In [23]:
beta_hat = solve(H, X.T @ (W * z), assume_a="pos")
beta_hat

array([7.41048576, 0.87259603])

### CN normalized model

In [173]:
#cn = np.log(cnv)
# CN-informed GLM
counts_init = counts/cnv # divide counts_tumor by cn_tumor & counts_normal by cn_normal only for inizialization 
y = np.log(counts_init / size_factors + 0.1)
y

array([6.893208  , 7.10323084, 7.59211192, 6.38430096, 5.6331186 ,
       4.69016757, 4.62667901, 7.329705  , 6.70657548, 5.28427134,
       8.84440181, 9.02970798, 7.28923426, 8.13832447, 7.8452454 ,
       8.59104244, 8.24332925, 7.59491508, 7.00109447, 8.26429162])

In [174]:
X_t = X.T
X_t

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [205]:
#cnv = [4, 5, 3, 4, 4, 3, 4, 4, 3, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
#cnv = np.array(cnv)
#cnv = cnv/2
#cnv

In [181]:
min_mu = 0.5
mu = np.maximum(size_factors * cnv * np.exp(X_t[0] * beta[0] + X_t[1] * beta[1]), min_mu) #modificare ordine del cnv
mu # controllare se devo passare log(cn) 
# passare +log(CN)

array([1.09677848e+06, 3.17047232e+07, 3.46602370e+04, 1.12008038e+06,
       1.02026448e+06, 3.54635534e+04, 1.09331970e+06, 7.29114466e+05,
       3.54065974e+04, 2.65914406e+07, 2.91196042e+03, 3.43796640e+03,
       2.93632733e+03, 3.03894710e+03, 2.55253545e+03, 3.23749656e+03,
       3.49166331e+03, 3.23622948e+03, 2.58769137e+03, 3.43050468e+03])

In [182]:
W = mu / (1.0 + mu * disp)
W

array([0.74336586, 0.74336635, 0.74335042, 0.74336587, 0.74336582,
       0.74335078, 0.74336586, 0.74336561, 0.74335076, 0.74336635,
       0.74317665, 0.74320567, 0.74317822, 0.74318457, 0.74314994,
       0.74319572, 0.74320814, 0.74319565, 0.74315288, 0.74320532])

In [183]:
z = np.log(mu / size_factors) + (counts - mu) / mu
z

array([12.75340715, 16.18923607,  9.4121513 , 12.75256884, 12.75190146,
        9.31889316, 12.75152322, 12.75455743,  9.35417778, 16.18914881,
        9.22292401,  9.65835486,  7.53574321,  8.13978745,  7.87161101,
        8.74423766,  8.25666577,  7.6972178 ,  7.42268437,  8.28150437])

In [184]:
H = (X.T * W) @ X + ridge_factor
H

array([[14.86546745,  7.43185276],
       [ 7.43185276,  7.43185376]])

In [185]:
beta_hat = solve(H, X.T @ (W * z), assume_a="pos")
beta_hat

array([12.42277355, -4.13969109])

### Test GLM

In [32]:
def nb_nll(
    counts: np.ndarray, mu: np.ndarray, alpha: Union[float, np.ndarray]
) -> Union[float, np.ndarray]:
    n = len(counts)
    alpha_neg1 = 1 / alpha
    logbinom = gammaln(counts + alpha_neg1) - gammaln(counts + 1) - gammaln(alpha_neg1)
    if hasattr(alpha, "__len__") and len(alpha) > 1:
        return (
            alpha_neg1 * np.log(alpha)
            - logbinom
            + (counts + alpha_neg1) * np.log(mu + alpha_neg1)
            - (counts * np.log(mu))
        ).sum(0)
    else:
        return (
            n * alpha_neg1 * np.log(alpha)
            + (
                -logbinom
                + (counts + alpha_neg1) * np.log(alpha_neg1 + mu)
                - counts * np.log(mu)
            ).sum()
        )

In [33]:
def vec_nb_nll(counts: np.ndarray, mu: np.ndarray, alpha: np.ndarray) -> np.ndarray:
    n = len(counts)
    alpha_neg1 = 1 / alpha
    logbinom = (
        gammaln(counts[:, None] + alpha_neg1)
        - gammaln(counts + 1)[:, None]
        - gammaln(alpha_neg1)
    )

    if len(mu.shape) == 1:
        return n * alpha_neg1 * np.log(alpha) + (
            -logbinom
            + (counts[:, None] + alpha_neg1) * np.log(mu[:, None] + alpha_neg1)
            - (counts * np.log(mu))[:, None]
        ).sum(0)
    else:
        return n * alpha_neg1 * np.log(alpha) + (
            -logbinom
            + (counts[:, None] + alpha_neg1) * np.log(mu + alpha_neg1)
            - (counts[:, None] * np.log(mu))
        ).sum(0)

In [34]:
def grid_fit_beta(
    counts: np.ndarray,
    size_factors: np.ndarray,
    cnv: np.ndarray,
    design_matrix: np.ndarray,
    disp: float,
    min_mu: float = 0.5,
    grid_length: int = 60,
    min_beta: float = -30,
    max_beta: float = 30,
) -> np.ndarray:
    x_grid = np.linspace(min_beta, max_beta, grid_length)
    y_grid = np.linspace(min_beta, max_beta, grid_length)
    ll_grid = np.zeros((grid_length, grid_length))

    def loss(beta: np.ndarray) -> np.ndarray:
        # closure to minimize
        design_matrix_t = design_matrix.T
        mu = np.maximum(size_factors[:, None] * np.exp(design_matrix_t[0] * beta[0] + design_matrix_t[1] * beta[1]), min_mu)
        #mu = np.maximum(size_factors[:, None] * np.exp(design_matrix @ beta), min_mu)
        return vec_nb_nll(counts, mu, disp) + 0.5 * (1e-6 * beta**2).sum(1)

    for i, x in enumerate(x_grid):
        ll_grid[i, :] = loss(np.array([[x, y] for y in y_grid]))

    min_idxs = np.unravel_index(np.argmin(ll_grid, axis=None), ll_grid.shape)
    delta = x_grid[1] - x_grid[0]

    fine_x_grid = np.linspace(
        x_grid[min_idxs[0]] - delta, x_grid[min_idxs[0]] + delta, grid_length
    )

    fine_y_grid = np.linspace(
        y_grid[min_idxs[1]] - delta,
        y_grid[min_idxs[1]] + delta,
        grid_length,
    )

    for i, x in enumerate(fine_x_grid):
        ll_grid[i, :] = loss(np.array([[x, y] for y in fine_y_grid]))

    min_idxs = np.unravel_index(np.argmin(ll_grid, axis=None), ll_grid.shape)
    beta = np.array([fine_x_grid[min_idxs[0]], fine_y_grid[min_idxs[1]]])
    return beta

In [35]:
cnv

array([1.00001, 1.00001, 1.00001, 1.00001, 1.00001, 1.00001, 1.00001,
       1.00001, 1.00001, 1.00001, 2.00001, 2.50001, 1.50001, 3.50001,
       2.00001, 4.00001, 3.00001, 2.00001, 3.00001, 2.50001])

In [36]:
def irls_solver(
    counts: np.ndarray,
    size_factors: np.ndarray,
    design_matrix: np.ndarray,
    cnv: np.ndarray,
    disp: float,
    min_mu: float = 0.5,
    beta_tol: float = 1e-8,
    min_beta: float = -30,
    max_beta: float = 30,
    optimizer: Literal["BFGS", "L-BFGS-B"] = "L-BFGS-B",
    maxiter: int = 250,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, bool]:

    assert optimizer in ["BFGS", "L-BFGS-B"]
    
    num_vars = design_matrix.shape[1]
    X = design_matrix
    
    # if full rank, estimate initial betas for IRLS below
    if np.linalg.matrix_rank(X) == num_vars:
        Q, R = np.linalg.qr(X)
        offset = size_factors + np.log(cnv)
        y = np.log(counts / offset + 0.1)
        #y = np.log(counts / size_factors + 0.1)
        beta_init = solve(R, Q.T @ y)
        beta = beta_init

    else:  # Initialise intercept with log base mean
        beta_init = np.zeros(num_vars)
        beta_init[0] = np.log(counts / offset).mean()
        #beta_init[0] = np.log(counts / size_factors).mean()
        beta = beta_init
        
    dev = 1000.0
    dev_ratio = 1.0

    ridge_factor = np.diag(np.repeat(1e-6, num_vars))
    #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)

    X_t = X.T
    #mu = np.maximum(size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1]), min_mu)
    mu = np.maximum(offset * np.exp(X @ beta), min_mu)
    
    converged = True
    i = 0
    while dev_ratio > beta_tol:
        W = mu / (1.0 + mu * disp)
        z = np.log(mu / offset) + (counts - mu) / mu
        #z = np.log(mu / size_factors) + (counts - mu) / mu
        H = (X.T * W) @ X + ridge_factor
        beta_hat = solve(H, X.T @ (W * z), assume_a="pos")
        i += 1

        if sum(np.abs(beta_hat) > max_beta) > 0 or i >= maxiter:
            # If IRLS starts diverging, use L-BFGS-B
            def f(beta: np.ndarray) -> float:
                # closure to minimize
                mu_ = np.maximum(offset * np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1]), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                
                return nb_nll(counts, mu_, disp) + 0.5 * (ridge_factor @ beta**2).sum()

            def df(beta: np.ndarray) -> np.ndarray:
                mu_ = np.maximum(offset * np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1]), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                return (
                    -X.T @ counts
                    + ((1 / disp + counts) * mu_ / (1 / disp + mu_)) @ X
                    + ridge_factor @ beta
                )

            res = minimize(
                f,
                beta_init,
                jac=df,
                method=optimizer,
                bounds=(
                    [(min_beta, max_beta)] * num_vars
                    if optimizer == "L-BFGS-B"
                    else None
                ),
            )

            beta = res.x
            #mu = np.maximum(offset * np.exp(X @ beta), min_mu)
            #mu = np.maximum(size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1]), min_mu)
            mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
            converged = res.success

            if not res.success and num_vars <= 2:
                beta = grid_fit_beta(
                    counts,
                    size_factors,
                    X,
                    disp,
                )
                #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu
                mu = np.maximum(size_factors * np.exp(X_t[0] * beta[0] + X_t[1] * beta[1]), min_mu) 
            break

        beta = beta_hat
        mu = np.maximum(offset * np.exp(X @ beta), min_mu)
        #mu = np.maximum(size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1]), min_mu)
        #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
        # Compute deviation
        old_dev = dev
        # Replaced deviation with -2 * nll, as in the R code
        dev = -2 * nb_nll(counts, mu, disp)
        dev_ratio = np.abs(dev - old_dev) / (np.abs(dev) + 0.1)

    # Compute H diagonal (useful for Cook distance outlier filtering)
    W = mu / (1.0 + mu * disp)
    W_sq = np.sqrt(W)
    XtWX = (X.T * W) @ X + ridge_factor
    H = W_sq * np.diag(X @ np.linalg.inv(XtWX) @ X.T) * W_sq
    # Return an UNthresholded mu (as in the R code)
    # Previous quantities are estimated with a threshold though
    mu = offset * np.exp(X @ beta)
    #mu = size_factors * np.exp(X_t[0] * beta[0] * cnv + X_t[1] * beta[1])
    #mu = size_factors * np.exp(X @ beta)

    print("Beta parameters:", beta), 
    print("Estimated mean:", np.array(mu)), 
    print("H:", np.array(H)),
    print("Convergence:", converged)
    
    return beta, mu, H, converged

In [56]:
# Classical GLM
irls_solver(counts, size_factors, design_matrix, disp)

Beta parameters: [7.42819011 1.85418179]
Estimated mean: [ 1967.99858081  1828.10357256  1935.38075053  2009.81021647
  1830.70609429  1980.23685096  1961.79232929  1308.2826297
  1977.05650339  1533.27021047  9650.78978847 11394.07347687
  9731.54634261 10071.64775884  8459.58717055 10729.67835561
 11572.035241   10725.47900749  8576.10056945 11369.34391072]
H: [0.10000348 0.10000059 0.10000284 0.10000426 0.10000064 0.10000371
 0.10000336 0.09998444 0.10000365 0.09999277 0.09999964 0.10000081
 0.0999997  0.09999996 0.09999855 0.10000041 0.10000091 0.10000041
 0.09999867 0.1000008 ]
Convergence: True


(array([7.42819011, 1.85418179]),
 array([ 1967.99858081,  1828.10357256,  1935.38075053,  2009.81021647,
         1830.70609429,  1980.23685096,  1961.79232929,  1308.2826297 ,
         1977.05650339,  1533.27021047,  9650.78978847, 11394.07347687,
         9731.54634261, 10071.64775884,  8459.58717055, 10729.67835561,
        11572.035241  , 10725.47900749,  8576.10056945, 11369.34391072]),
 array([0.10000348, 0.10000059, 0.10000284, 0.10000426, 0.10000064,
        0.10000371, 0.10000336, 0.09998444, 0.10000365, 0.09999277,
        0.09999964, 0.10000081, 0.0999997 , 0.09999996, 0.09999855,
        0.10000041, 0.10000091, 0.10000041, 0.09999867, 0.1000008 ]),
 True)

In [37]:
# CN normalized GLM
irls_solver(counts, size_factors, design_matrix, cnv, disp)

Beta parameters: [7.42812371 0.27144301]
Estimated mean: [1967.88474121 1827.99902134 1935.26907659 2009.69360077 1830.60137027
 1980.12219879 1961.67890175 1308.21259218 1976.94206237 1533.18523481
 3512.31212302 4362.94223821 2893.87612598 4834.04187735 3267.6413555
 5263.95537222 4801.94880507 3733.0512585  4186.58948289 4357.86283204]
H: [0.10000779 0.10000135 0.10000637 0.10000954 0.10000148 0.10000831
 0.10000752 0.0999654  0.10000817 0.09998395 0.09999436 0.10000355
 0.09998429 0.10000725 0.09999083 0.10001004 0.10000702 0.09999715
 0.10000195 0.1000035 ]
Convergence: True


(array([7.42812371, 0.27144301]),
 array([1967.88474121, 1827.99902134, 1935.26907659, 2009.69360077,
        1830.60137027, 1980.12219879, 1961.67890175, 1308.21259218,
        1976.94206237, 1533.18523481, 3512.31212302, 4362.94223821,
        2893.87612598, 4834.04187735, 3267.6413555 , 5263.95537222,
        4801.94880507, 3733.0512585 , 4186.58948289, 4357.86283204]),
 array([0.10000779, 0.10000135, 0.10000637, 0.10000954, 0.10000148,
        0.10000831, 0.10000752, 0.0999654 , 0.10000817, 0.09998395,
        0.09999436, 0.10000355, 0.09998429, 0.10000725, 0.09999083,
        0.10001004, 0.10000702, 0.09999715, 0.10000195, 0.1000035 ]),
 True)